In [7]:
import numpy as np

import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web
pd.options.display.float_format = "{:,.2f}".format

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-white")
plt.rcParams["font.size"] = 11
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["figure.dpi"] = 1500
plt.rcParams["figure.figsize"] = (15,6)

from datetime import datetime

import plotly.graph_objs as go
import plotly as py

import yfinance as yf

# momentum indicators
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import StochasticOscillator
from ta.momentum import WilliamsRIndicator

# trend indicators
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.trend import WMAIndicator
from ta.trend import SMAIndicator

# volatility indicators
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# volume indicators
from ta.volume import money_flow_index
from ta.volume import OnBalanceVolumeIndicator

In [8]:
# sklearn libraries
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle

from sklearn.svm import SVR

from sklearn.model_selection import (train_test_split,
                                     cross_validate,
                                     GridSearchCV
                                    )

from sklearn.metrics import (f1_score, 
                             accuracy_score, 
                             classification_report, 
                             confusion_matrix
                            )

# from sklearn import neural_network

In [9]:
import m as a    # import module, file/download as/python(.py)

In [10]:
end = datetime.today()
start = datetime(end.year-2, end.month, end.day)
ticker = web.DataReader("^GSPC", "yahoo", start, end)

In [11]:
t = a.TechnicalIndicators(ticker["Close"], ticker["High"], ticker["Low"], ticker["Volume"])
t.set_technical_indicators()

if __name__ == "__main__":
    t.rsi()
    t.rate_change()
    t.stochastic_oscillator()
    t.WilliamsR()

    t.macd()
    t.ema()
    t.wma()
    t.moving_average()

    t.bollinger_bands()
    t.average_true_range()

    t.on_balance_volume()
    t.money_flow_index()

In [ ]:
class SupportVectorMachines():
    """
    This is an example of the support vector machines class.
    """
    def __init__(self, 
                 C = 1.0, 
                 kernel = "rbf", 
                 gamma = 1.0, 
                 max_iterations = 10000, 
                 random_state = None):
        
        self.C = C
        self.kernel = kernel
        self.gamma = gamma
        self.max_iterations = max_iterations
        self.random_state = random_state

In [ ]:
# Stock_df = pd.DataFrame()
# Stock_df['DateTime'] = Page_Data['chart']['result'][0]['timestamp']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.9)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

param_grid = {'C': [ 1, 10, 100, 1000,10000], 
              'gamma': [1,0.1,0.01,0.001,0.0001],
              'kernel': ['rbf']}

def Split_Train_Test(data, test_ratio):
    train_set_size = 1 - int(len(data) * test_ratio)
    train_set = data[:train_set_size]
    test_set = data[train_set_size:]
    return train_set, test_set

scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)